In [ ]:
# Let's do some setup

# Install pytorch gpu

# Use the command below to install pytorch in gpu
#!pip3 install torch torchvision torchaudio

# Use command below to install pytorch in cpu
!pip3 install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cpu


Looking in indexes: https://download.pytorch.org/whl/cpu


In [ ]:
# unzip imagenette

In [ ]:
%cd /content/data
!ls
#!wget https://s3.amazonaws.com/fast-ai-imageclas/imagenette2.tgz

/content/data
class_code2name.json  class_id2name.json  imagenet_patch.gz
--2023-12-05 13:06:58--  https://s3.amazonaws.com/fast-ai-imageclas/imagenette2.tgz
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.216.216.224, 52.216.43.112, 52.216.61.144, ...
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.216.216.224|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1557161267 (1.5G) [application/x-tar]
Saving to: ‘imagenette2.tgz’

imagenette2.tgz     100%[===================>]   1.45G  23.3MB/s    in 56s     

2023-12-05 13:07:55 (26.4 MB/s) - ‘imagenette2.tgz’ saved [1557161267/1557161267]



In [20]:
!python index.py --operation attack

program started for operation attack
building datasets...

Length of train dataset:  9296
Length of val dataset:  3856 

Reset output directories

Loading pretrained model

Attack images
	Adding patches to train batch 0
	Adding patches to train batch 1
	Adding patches to train batch 2
	Adding patches to train batch 3
	Adding patches to train batch 4
	Adding patches to train batch 5
	Adding patches to train batch 6
	Adding patches to train batch 7
	Adding patches to train batch 8
	Adding patches to train batch 9
	Adding patches to train batch 10
	Adding patches to train batch 11
	Adding patches to train batch 12
	Adding patches to train batch 13
	Adding patches to train batch 14
	Adding patches to train batch 15
	Adding patches to train batch 16
	Adding patches to train batch 17
	Adding patches to train batch 18
	Adding patches to train batch 19
	Adding patches to train batch 20
	Adding patches to train batch 21
	Adding patches to train batch 22
	Adding patches to train batch 23
	Adding

In [21]:
!python make_yolov5_dataset.py

No of images copied 13152 11826 1326
['.config', '__init__.py', '.ipynb_checkpoints', 'models', 'transforms', 'test-mask-2.png', 'README.md', 'test_img.JPEG', 'datautils', 'utils', 'output', 'test-mask.png', 'make_yolov5_dataset.py', 'yolov5', 'preprocessor.py', 'train_loss_history.png', 'test-mask-3.png', 'attack', 'sample_patches.png', 'index.py', 'test-attacked-images', 'yolov5-attacked-images.zip', 'defence', 'config.py', 'data', 'runs', 'patch-detection.yaml', '__pycache__', 'attacked-images.zip', 'sample_data']
/content/data/yolov5-attacked-images /content/yolov5/data/yolov5-attacked-images


In [ ]:
!python yolov5/train.py --img 224 --batch 32 --epochs 50 --data yolov5/data/patch-detection.yaml --weights yolov5s.pt --workers 24 --name yolo-patch-detection-1

train: weights=yolov5s.pt, cfg=, data=yolov5/data/patch-detection.yaml, hyp=yolov5/data/hyps/hyp.scratch-low.yaml, epochs=50, batch_size=32, imgsz=224, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, bucket=, cache=None, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=24, project=yolov5/runs/train, name=yolo-patch-detection-1, exist_ok=False, quad=False, cos_lr=False, label_smoothing=0.0, patience=100, freeze=[0], save_period=-1, seed=0, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
github: up to date with https://github.com/ultralytics/yolov5 ✅
YOLOv5 🚀 v7.0-247-g3f02fde Python-3.10.12 torch-2.0.1+cu118 CUDA:0 (Tesla T4, 15102MiB)

hyperparameters: lr0=0.01, lrf=0.01, momentum=0.937, weight_decay=0.0005, warmup_epochs=3.0, warmup_momentum=0.8, warmup_bias_lr=0.1, box=0.05, cls=0.5, cls_pw=1.0, obj=1.0, obj_pw=1.0, iou_t=0.2, anchor_t

In [25]:
!python yolov5/detect.py --weights yolov5/runs/train/yolo-patch-detection-1/weights/best.pt --img 224 --save-txt --source data/yolov5-attacked-images/val/images

detect: weights=['yolov5/runs/train/yolo-patch-detection-1/weights/best.pt'], source=data/yolov5-attacked-images/val/images, data=yolov5/data/coco128.yaml, imgsz=[224, 224], conf_thres=0.25, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=True, save_csv=False, save_conf=False, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=yolov5/runs/detect, name=exp, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False, vid_stride=1
YOLOv5 🚀 v7.0-247-g3f02fde Python-3.10.12 torch-2.0.1+cu118 CUDA:0 (Tesla T4, 15102MiB)

Fusing layers... 
Model summary: 157 layers, 7015519 parameters, 0 gradients, 15.8 GFLOPs
image 1/3856 /content/data/yolov5-attacked-images/val/images/ILSVRC2012_val_00000732.JPEG: 224x224 1 patch, 7.6ms
image 2/3856 /content/data/yolov5-attacked-images/val/images/ILSVRC2012_val_00001432.JPEG: 224x224 1 patch, 5.3ms
image 3/3856 /content/data/yolov5-attacked-i

In [29]:
!python index.py --operation defence --test_dir yolov5/runs/detect/exp4/labels

program started for operation defence
building datasets...

Length of train dataset:  128
Length of val dataset:  51 


Loading pretrained model
experiment to be evaluated /content/yolov5/runs/detect/exp4/labels

Clean image accuracy:  53.0
Attacked image accuracy:  48.92156862745098
Defence image accuracy:  35.705882352941174
